<div class="report-header"><div class="aictx-logo"></div>
<span class="report-type">Documentation</span><br />
<span class="report-author">Author: Felix Bauer</span><br />
<span class="report-date">7th January, 2019</span>
</div><h1>DynapseControl:</h1><h1>Interface for cortexcontrol</h1>

This document illustrates how to use the `DynapseControl` class to interface `cortexcontrol` and the DynapSE processor.

##### Housekeeping and import statements

In [1]:
# - Initialisation code to include custom report styles
from IPython.core.display import HTML
def css_styling():
    styles = open("aictx-report.css", "r").read()
    return HTML(styles)
css_styling()

The `DynapseControl` class provides higher-level functionality for `cortexcontrol` and allows the `NetworksPython` framework to interact with the DynapSE processor. However, it can also be used from outside `NetworksPython`.

<br>

## Setup

<br>

### Connecting to Cortexcontrol

In order to work interface the DynapSE chip, this layer relies on `Cortexcontrol`. It can either be used from within the `Cortexcontrol` console or via an `RPyC` connection. In order to run some examples from within this jupyter notebook, we will do the latter. For this we start `Cortexcontrol` and run the following commands in its console (not in this notebook):

If the `cortexcontrol` console prints <br>
<i>"RPyC: Ready to start."</i> <br>
and nothing else, it is ready. Now through your usual python setup (console, iPython, jupyter, ...) you can import the class and instantiate a `DynapseControl` object.

<br>

### Import and class  instantiation

The `DynapseControl` class can be imported from the `NetworksPython` package and is defined in the `dynapse_control` module. Objects can be instantiated without any arguments:


In [2]:
# - Import class
from NetworksPython import DynapseControl

# - Easy instantiation
dc = DynapseControl()

/home/felix/miniconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


dynapse_control: RPyC connection established.
dynapse_control: CtxDynapse modules loaded.
dynapse_control: Initializing hardware.
dynapse_control: Hardware has already been initialized.
DynapseControl: Initializing DynapSE
DynapseControl: Spike generator module ready.
DynapseControl: Time constants of cores range(0, 16) have been reset.
DynapseControl: Neurons initialized.
	 4092 hardware neurons and 4092 virtual neurons available.
DynapseControl: Neuron connector initialized
DynapseControl: Connections to cores None have been cleared.
DynapseControl: FPGA spike generator prepared.
DynapseControl ready.


There are two optional arguments:

| Argument | Type | Default | Meaning |
|----------|------|---------|---------|
| `tFpgaIsiBase` | `float`| `2e-5` | Time step for sending data to hardware |
| `lnClearCores` | `list` or `None` | `None` | IDs of cores whose configuration should be cleared. |

`tFpgaIsiBase` is a `float` determines the time step size of the inter-spike intervals between events that are sent to the processor. For more details, see <i>FPGA</i>.

`lnClearCores` can either be `None` or a list of integers. In the latter case the configurations of cores with corresponding IDs are wiped, so that  previously set synaptic connections, that the user may not aware of, are removed. To be more precise, the CAMs (pre-neuron IDs and pre-neuron core IDs) and the SRAMs (target chip ID, virtual core ID, core mask) are cleared. `lnClearCores` should include the IDs of all cores that you are going to work on. In order to save some time, you can omit the others. In case of an empty list or `None` as argument, no cores are cleared.

<br>

## Hardware

Before we go into the details of how the class works, let us first have a brief look at some properties of the DynapSE.

The DynapSE processor consists of 4 chips. Each chips has 4 cores of 256 neurons. The chips, as well as each core in a chip and each neuron in a core are identified with an ID between 0 and 3 or 0 and 256, respectively. However, this class uses logical IDs from 0 to 4095 that range over all neurons. In other words the logical neuron ID is $1024 \cdot \text{ChipID} + 256 \cdot \text{CoreID} + \text{NeuronID}$.

<br>

### Setting connections

#### Synapse types
There are four different synapse types on the DynapSE: fast and slow excitatory as well as fast and slow inhibitory. Each neuron can receive inputs through up to 64 synapses, each of which can be any of the given types. Via `cortexcontrol` the synaptic behavior can be adjusted for each type and for each core, but not for individual synapses.

There is a priori no difference between slow and fast excitatory synapses, so they can be set to have the same behavior. In fact, one could assign shorter time constants to the "slow" excitatory synapses, making them effectively the fast ones. While both excitatory and the fast inhibitory synapses work by adding or subtracting current to the neuron membrane, the slow inhibitory synapses use shunt inhibition and in practice silence a neuron very quickly.

Note that all synapses that are of the same type and that are on the same core have the same weight. Different connection strengths between neurons can only be achieved by setting the same connection multiple times. 

<br>

#### Understanding the routing scheme
In particular if you want to implement complex or large networks that span over several cores, it is helpful to understand how connections are set on the DynapSE. Each neuron has 4 SRAM and 64 CAM cells. 

##### SRAM
The SRAM cells determine where each neuron sends its output events, in other words, this will affect postsynaptic connections of the neuron. You can specify the following parameters:
- `target_chip_id` $\in [0, 4)$: ID of chip to which events are sent.
- `core_mask` $\in [0, 16)$: This value is converted to a 4-digit binary value, where each digit corresponds to one core and determines if this core receives events (1) or not (0).
- `virtual_core_id` $\in [0, 4)$: Neurons receive events from specific cores (see below). If this value is changed, the sending neuron pretends that it belongs to a different core than it actually does.
Because each neuron has 4 SRAM cells it can send events to up to 4 chips.

##### CAM
In each CAM cell the neuron can be set to listen to (i.e. receive events from) one other neuron. In other words, presynaptic connections are set here. The following parameters can be set:
- `pre_neuron_ID` $\in [0, 256)$: The presynaptic neuron's ID within its core.
- `pre_neuron_core_ID` $\in [0, 1024)$: The core of the presynaptic neuron.

These two values together uniquely define a presynaptic neuron on a single chip. From which chips the postsynaptic neuron receives the events is determined by which neurons (with corresponding ID and core) send to the core where the postsynaptic neuron is located (i.e. the presynaptic neurons must have matching `target_chip_id` and `core_mask` in their SRAMs (see above).
- `type`: The synapse type of this connection.
The `type`s are `DynapseCamType` objects that can be accessed from the `dynapse_control` module as: `SynapseTypes.<type>` where `<type>` is `FAST_EXC`, `SLOW_EXC`, `FAST_INH` or `SLOW_INH`. Using the `DynapseControl` class these synapse typse can be accessed as `DynapseControl.synFE`, `DynapseControl.synSE`, `DynapseControl.synFI` and  `DynapseControl.synSI`, respectively.
Becuase each neuron has 64 CAM cells, it can have at most 64 presynaptic connections.

<br>

### Sending events

#### Virtual neurons
Inputs are sent to the hardware through so-called virtual neurons. Every virtual neuron on the DynapSE has a logical ID, which ranges from 0 to 1023.  An event that is sent to the hardware translates into a virtual neuron emitting a spike.

If a physical neuron $m$ is set to receive spikes from a neuron with logical ID $n$, it makes no difference if a virtual or a pysical neuron with this ID is firing (as long as the firing neuron's target chip IDs and core masks include the location $m$). Neuron $m$ will receive the spikes in both cases. Because this can cause unexpected behavior it is recommended that the IDs of the physical neurons are pairwise different from the IDs of of the virtual neurons.

#### FPGA
As for now it is not possible with `DynapseControl` to stream events continuously to the DynapSE. Instead, events are first sent to an FPGA, temporarily stored there and then translated to spikes of virtual neurons, with temporal order and inter-spike intervals matching the input signal. The number of events that can be sent at once is limited to $2^{16} - 1 = 65535$. Longer inputs need to be split up.

Inter-spike intervals must be multiples of a time step, which in `DynapseControl` is stored as `tFpgaIsiBase`. This time step, in return,  needs to be a multiple of $11.\overline{1} \cdot 10^{-9} \text{ s}$.

Although the size of `tFpgaIsiBase` has no effect on computation time, it is not always advisable to choose the smallest possible value. The reason is that the number of timesteps between two input spikes is limited to $2^{16}-1 = 65535$. This means that with `tFpgaIsiBase` $= 11.\overline{1} \cdot 10^{-9} \text{ s}$, any inter-spike interval longer than about 0.73 milliseconds is already above the limit. `DynapseControl` will automatically reduce inter-spike intervals that are too long by inserting "dummy" events that are not routed to any neuron on the chip. However, if too many are inserted, the maximum number of input events can be exceeded and the input will have to be split up. In this case a `MemoryError` will be raised.

Therefore it makes sense to set `tFpgaIsiBase` to something between $10^{-6}$ and $10^{-4}$ seconds, in order to allow for sufficiently long silent parts in the input while still maintaining a good temporal resolution. Alternatively one could send dummy events that do not target any physical neuron. However, transmission of events to the processor is not the fastest and the more events are sent, the longer it takes.

<br>

### Receiving events
In order to record the firing activity of neurons `EventFilter` and `BufferedEventFilter` objects can be used. Each can be set to listen to a specific population of neurons. `EventFilter`s will call a callback function, each time one of the specified neurons fires. `BufferedEventFilter`s will store `event` objects in a buffer until they are fetched. For more details refer to the <a href="http://ai-ctx.gitlab.io/ctxctl/primer.html#CtxDynapse.EventFilter">cortexcontrol documentation</a>.

Currently the `DynapseControl` class only supports the use of a single `BufferedEventFilters`, which can be added or changed with the `add_buffered_filter` method. It will stop recording when the `clear_buffered_filter` method is called.

<font color="red">
- EventFilter and BufferedEventFilter
</font>

### Time constants for neurons
For each core the neuron time constants can be set through the biases `IF_TAU1_N` and `IF_TAU2_N`. Higher values correspond to shorter time constants and effectively make it less likely that the neurons will fire. For each neuron you can choose which of the two time constant biases applies. This can be useful, for example by setting the value of `IF_TAU2_N` very high, making it almost impossible for a neuron to fire. Neurons that fire in an uncontrollable way (so-called hot neurons) can then be silenced by assigning them `TAU2` instead of `TAU1`.
Neurons are assigned `TAU2` with the `silence_neurons` method. All neurons of specified cores are assigned `TAU1` with the `reset_silencing` method. Note that the names of these methods are chosen under the assumption that the bias for `TAU2` has been set to a high value. The neurons are not silenced per se by those methods.


### Neuron allocation

### Shadow state neurons

<br>

## Methods overview
Now that a `DynapseControl` has been instantiated, there are many ways of using it to interact with the hardware. This section will give an overview over all mehtods that are currently implemented. For more details please consult the documentation within the source code. 

<br>

### Neuron allocation and connections

`allocate_hw_neurons`<br>
Allocate neurons if available and return them in lists.

`allocate_virtual_neurons`<br>
Allocate virtual neurons if available and return them in list.

`connect_to_virtual`<br>
Connect populations of virtual and physical neurons.

`set_virtual_connections_from_weights`<br>
Set connections from virtual to physical neurons from weight matrix.

`set_connections_from_weights`<br>
Set connections between physical neurons from weight matrix.

`remove_all_connections_to`<br>
Remove all presynaptic connections to specified neurons.

<br>

### Resetting

`clear_connections` <br>
Reset connections for specified cores.

`reset_silencing` <br>
Assign time constant `TAU1` to all neurons on specified cores.

`reset_cores` <br>
Reset neuron assignments, time constants and presynaptic connections for specified cores.

`reset_all` <br>
Reset neuron assignments, time constants and presynaptic connections for all cores.

<br>

### Stimulation and event generation
`start_cont_stim`<br>
Start sending events to processor with fixed frequency.

`stop_stim`<br>
Stop any ongoing stimulation except poisson-rate stimulation.

`start_poisson_stim`<br>
Start generating events by poisson processes and send them.

`stop_poisson_stim`<br>
Stop generating events by poisson processes and send them.

`reset_poisson_rates`<br>
Set rates for poisson event generation to 0.

`send_pulse`<br>
Send a pulse of periodic input events.

`send_TSEvent`<br>
Extract events from a `TSEvent` object and send them to the processor. Can record neuron activity simultaneously.

`send_arrays`<br>
Extract events from arrays (times and channels) and send them to the processor. Can record neuron activity simultaneously.

<br>

### Tuning and observing activities

`add_buffered_event_filter`<br>
Add a `BufferedEventFilter` to record activities from specified neurons.

`clear_buffered_event_filter`<br>
Stop recording neuron activities.

`collect_spiking_neurons`<br>
Return a list of neurons that spike during a specified time.

`silence_hot_neurons`<br>
Set time constant to `TAU2` for neurons that spike during specified time.

`measure_population_firing_rates`<br>
Measure firing rates for multiple neuron populations.

`measure_firing_rates`<br>
Measure firing rates of specified neurons.

`monitor_firing_rates`<br>
Start periodically printing the average firing rates for specified neurons.

`stop_monitor`<br>
Stop measuring and printing firing rates.

`sweep_freq_measure_rate`<br>
Stimulate a group of neurons by sweeping over a list of input frequencies. Measure their firing rates.

<br>

### Load and save biases

`load_biases`<br>
Load biases from file.

`save_biases`<br>
Save biases to file.

`copy_biases`<br>
Copy biases from one core to other(s).

<br>

## Using RPyC

This section contains useful information if you want to contribute own methods for interacting with the processor. For this we will use a few objects from the `dynapse_control`, in which also the `DynapseControl` class is defined. We will also import the `time` function from the eponymous built-in package to do compare efficiency of different approaches.

In [2]:
# - Import NetworksPython.dynapse_control and built-in time package
from NetworksPython import dynapse_control as DC
from time import time

/home/felix/miniconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


ImportError: cannot import name 'dynapse_control' from 'NetworksPython' (/home/felix/gitlab/network-architectures/NetworksPython/__init__.py)

In order to communicate with `cortexcontrol`, the `dynapse_control` module uses the `RPyC` package (https://rpyc.readthedocs.io/en/latest/), which establishes a connection where `cortexcontrol` acts as a server that can execute python commands sent by the `dynapse_control` module. 
<br>

#### Adding objects to `cortexcontrol` namespace
It is important to note that `cortexcontrol` uses its own python interpreter with its own namespace. Therefore you need to make sure that objects and modules that are to be used there are properly defined. The `dynapse_control` module has a dictionary-like object called `conn.namespace`, where you can add items in order to add an object to the namespace of `cortexcontrol`, like in the following example:

In [1]:
# - Define some object
lNumbers = [1,2,3,4,5]
# - Make it accessible in the namespace of cortexcontrol
DC.conn.namespace["lNumbers"] = lNumbers

NameError: name 'DC' is not defined

#### Using built-in python modules
You can use built-in python modules such as `copy`, `time`, etc. within the `cortexcontrol` environment. For this you need to add `conn.modules.<built-in name>` to `conn.namespace`:

In [ ]:
# - Add copy module to cortexcontrol namespace
DC.conn.namespace["copy"] = DC.conn.modules.copy

### Pitfalls

#### Non-built-in objects
 no additional packages available other than `RPyC`. This means it will not be able to run commands that use non-built-in packages. Even though sometimes it can process corresponding objects, such as `numpy.ndarray`s, it is in general safer to convert such objects into similar built-in objects, such as lists. 

#### Data transfer
Repeatedly sending even small chunks of data through the `RPyC` connection takes a lot of time and should be avoided. This can happen for instance in loops that use objects on the other side of the connections. For example, a function in `dynapse_control` that loops over objects from the `cortexcontrol` environment, such as neurons, will be very inefficient:

In [12]:
t0 = time()

## -- Inefficient, avoid such loops:
# - List of neurons - lives on other side of connection
lNeurons = dc.model.get_neurons()
# - List of neuron IDs
lIDs = []
for neuron in lNeurons:
    # - At each iteration, data will be transferred to this side of the connection
    lIDs.append(neuron.get_id())
print("Took {} s".format(time()-t0))

Took 2.399355173110962 s


A better solution in this case would be to define a function that lives on the `cortexcontrol`-side (using the `@teleport_function` or `@remote_function` decorators - more details below) and generates the list there before sending it our side:

In [13]:
## -- Better: Process objects locally, send data only once

# - Define function on other side of connection wiht teleport_function decorator
@DC.teleport_function
def get_neuron_ids():
    lNeurons = CtxDynapse.model.get_neurons()
    # - List of neuron IDs
    lIDs = []
    for neuron in lNeurons:
        # - Data remains on one side of connection
        lIDs.append(neuron.get_id()) 
    return lIDs

t0 = time()

lIDs = get_neuron_ids()

print("Took {} s".format(time()-t0))

Took 0.020653963088989258 s


Be careful with function arguments and returns - an object will live on the side of the connection where it has been created. Use the `copy` module (or `deepcopy` for nested objects) to make sure, all data is transmitted to where you want to work with it. 

You can always see where an object lives by using `print(type(<object>))`. If the object is on the other side of the connection, its type will contain `netref`. This is always relative to the place where the command is called, so if it is called from within a funciton on the `cortexcontrol`-side and the output contains `netref`, the object is on the `dynapse_control`-side. A print statement called within `cortexcontrol` will print into the `cortexcontrol`-console.

In [23]:
import copy

# - List still lives in cortexcontrol. Looping over it would be inefficient.
lIDs = get_neuron_ids()
print(type(lIDs))
# - Copy it to this side 
lIDs = copy.copy(lIDs)
print(type(lIDs))

<netref class 'builtins.list'>
<class 'list'>


Be particularly careful with container objects, such as lists. It may happen that the container lives on a different side of a connection than its contained elements. This will inevitably cause performance issues.

One especially perfidious case are numbers that originate from `numpy` functions. They are often `numpy`-specific types such as `np.int64`. Even if you convert an array containing such numbers to a list, they will be of this special type. When this list is sent to a function on the other side of the loop, they will remain on your side of the connection, and `copy` or `deepcopy` on the `cortexcontrol`-side can't handle them. You should therefore loop over them and convert them to normal `int`s or `float`s. We will see below that there is a way to do this (semi-) automatically. The following code cell will throw an execption while the next one works.

In [29]:
# - Print type of elements in lNumbers in cortexcontrol-console
@DC.teleport_function
def print_element_type(lNumbers: list):
    print(type(lNumbers[0]))

# - Try using (deep-)copy on lNumbers
@DC.teleport_function
def print_element_type_copy(lNumbers: list):
    # - List is on correct side
    print(type(lNumbers))
    # - ...its elements not necessarily
    print(type(lNumbers[0]))
    # - (deep-)copy does not help
    lNumbers = copy.copy(lNumbers)
    # lNumbers = copy.deepcopy(lNumbers)
    print(type(lNumbers[0]))

# - Elements in lNumbers are not normal integers
lNumbers = list(np.random.randint(10, size=5))
print(type(lNumbers[0]))

print_element_type(lNumbers)  # - Will print type of first element in lNumbers in cortexcontrol-console.
                              #   Not that it is a netref.
print_element_type_copy(lNumbers)  # - Will throw an exception

<class 'numpy.int64'>


ModuleNotFoundError: No module named 'numpy'

========= Remote Traceback (1) =========
Traceback (most recent call last):
  File "/home/felix/cortexcontrol/python/site-packages/rpyc/core/protocol.py", line 329, in _dispatch_request
    res = self._HANDLERS[handler](self, *args)
  File "/home/felix/cortexcontrol/python/site-packages/rpyc/core/protocol.py", line 590, in _handle_call
    return obj(*args, **dict(kwargs))
  File "<ipython-input-29-8db39063a536>", line 15, in print_values_copy
  File "/home/felix/cortexcontrol/python/lib/python3.7/copy.py", line 106, in copy
    return _reconstruct(x, None, *rv)
  File "/home/felix/cortexcontrol/python/lib/python3.7/copy.py", line 274, in _reconstruct
    y = func(*args)
ModuleNotFoundError: No module named 'numpy'


In [30]:
# - Convert elements to normal integers
lNumbers = [int(n) for n in lNumbers]
print_values(lNumbers)  # - Will print type of new elements in cortexcontrol-console

<b>In summary, there are three rules that you should always respect:</b>
1. Do not use non-built-in types on the cortexcontrol side of the connection.
2. Avoid loops containing objects on the other side of the connection.
3. Copy data to the corresponding side before processing them. Be careful with nested types.

In the following we will see a few tools that help complying with these rules.

### Defining objects and functions on cortexcontrol-side

<font color="red">
    
- Difference between where object lives and where it is defined.
- How to include objects, modules and functions in cortexcontrol-namespace

- Define functions on cortexcontrol-side (teleport_function, remote_function)

- Make sure objects (function arguments) are only built-in types
- Make sure objects (function arguments) live on correct side of connection
</font>


#### Colophon
Live notebook requires a Jupyter Notebook server.

GitLab repository location: https://gitlab.com/ai-ctx/network-architectures/blob/master/Projects/Documentation/FFCLIAF%20and%20RecCLIAF.ipynb